# Run the notebook with : jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

In [ ]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
%run ../tasking_manager_stats/data_management

In [ ]:
start_date = datetime.datetime.strptime('2020-11-23 18:00:00', '%Y-%m-%d %H:%M:%S')
start_date -= datetime.timedelta(hours=1) # Winter
timestamps = pd.date_range(start_date, start_date + datetime.timedelta(hours=2), freq='5min')
timestamps

In [ ]:
project_id = 9849
db = Database(project_id)

In [ ]:
def compute_task_states(task_data, timestamps):
    task_states = np.zeros(len(timestamps))
    task_locked = np.zeros(len(timestamps))
    for task_hist in reversed(task_data['taskHistory']):
        date = pd.to_datetime(task_hist['actionDate'])
        timestamp = (date - timestamps[0]).days
        if task_hist['action'].startswith('LOCK'):
            task_locked[date < timestamps] = 1
            task_locked[date + pd.to_timedelta(task_hist['actionText']) < timestamps] = 0
            continue
        if task_hist['action'] != 'STATE_CHANGE':
            continue
        if task_hist['actionText'] == 'MAPPED':
            task_states[date < timestamps] = 1
            continue
        if task_hist['actionText'] == 'INVALIDATED':
            task_states[date < timestamps] = 2
            continue
        if task_hist['actionText'] == 'VALIDATED':
            task_states[date < timestamps] = 3
            continue
        if task_hist['actionText'] == 'BADIMAGERY':
            task_states[date < timestamps] = 4
            continue
    return task_states, task_locked

In [ ]:
def get_task_states_and_locked_tasks(db, timestamps):
    tasks_states = dict()
    tasks_locked = dict()
    for task_id in db.get_task_ids():
        task_states, task_locked = compute_task_states(db.get_task_history()[str(task_id)], timestamps)
        tasks_states[task_id] = task_states
        tasks_locked[task_id] = task_locked
    return tasks_states, tasks_locked

In [ ]:
tasks_states, tasks_locked = get_task_states_and_locked_tasks(db, timestamps)

In [ ]:
for feature in db.get_task_features():    
    arr = np.array(feature['geometry']['coordinates'][0][0]).transpose()
    task_id = feature['properties']['taskId']
    if task_id == 40:
        break
fig = go.Figure()
fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='#ADE6EF'), mode='lines', fill="toself"))
fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='black'), mode='lines', name=''))
fig.add_layout_image(dict(source="../data/lock.png",x=arr[0][:-1].mean(), y=arr[1][:-1].mean()))
fig.update_layout_images(dict(xref="x", yref="y", sizex=arr[0].max() - arr[0].min(), sizey=arr[1].max() - arr[1].min(),
                              xanchor="center", yanchor="middle"))
fig.show()

In [ ]:
time_id = -1
fig = go.Figure()
# Plot state and lock
for feature in db.get_task_features():    
    arr = np.array(feature['geometry']['coordinates'][0][0]).transpose()
    task_id = feature['properties']['taskId']
    state = tasks_states[task_id][time_id]
    if state == 1: # MAPPED
        fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='#ADE6EF'), mode='lines', fill='toself', name=''))
    elif state == 2: # INVALIDATED
        fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='#FCECA4'), mode='lines', fill='toself', name=''))
    elif state == 3: # VALIDATED
        fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='#40AC8C'), mode='lines', fill='toself', name=''))
    elif state == 4: # BADIMAGERY
        fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='#D8DAE4'), mode='lines', fill='toself', name=''))
        
    # Add lock if there is one
    if tasks_locked[task_id][time_id] == 1:
        fig.add_layout_image(dict(source="../data/lock.png",x=arr[0][:-1].mean(), y=arr[1][:-1].mean()))
        fig.update_layout_images(dict(xref="x", yref="y", sizex=arr[0].max() - arr[0].min(),
                                      sizey=arr[1].max() - arr[1].min(), xanchor="center", yanchor="middle"))
        
# Plot borders
for feature in db.get_task_features(): 
    arr = np.array(feature['geometry']['coordinates'][0][0]).transpose()
    fig.add_trace(go.Scatter(x=arr[0], y=arr[1], marker=dict(color='#999DB5'), mode='lines', name=''))   
fig.update_layout(showlegend=False, plot_bgcolor='rgba(0,0,0,0)', xaxis=dict(visible=False), yaxis=dict(visible=False))
fig.update_yaxes(scaleanchor = "x",scaleratio = 1)
fig.show()